# FODS Assignment 1

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("./data.csv")

In [8]:
df.head()

,X,Y
0,0.987988,5.098368
1,0.719720,2.516654
2,-0.403403,0.337961
3,0.107107,0.737320
4,0.345345,-0.780955


In [15]:
mean_y = df['Y'].mean()
std_y = df['Y'].std()

In [23]:
mean_y

0.9547432925098046

In [24]:
std_y

1.8091349909170027

In [16]:
df['Y'] = (df['Y'] - mean_y) / std_y

In [19]:
df.tail()

,X,Y
995,0.671672,1.174380
996,-0.615616,-0.512541
997,0.259259,0.062779
998,0.119119,-0.392207
999,0.369369,-0.226423


In [20]:
mean_t = df['Y'].mean()
std_t = df['Y'].std()

In [21]:
mean_t

1.7763568394002505e-18

In [22]:
std_t

1.0